In [2]:
import pandas as pd
from datetime import date, timedelta

def get_date_prompt(): #Asks the user for a date input, which can be a specific date, yesterday, today, or 'quit' to exit the application.
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        return input_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")

        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            return yesterday
        elif date_input == "2":
            return today
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date): # Asks the user for earnings for a specific date.
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                return float(day_earning)
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def generate_weekly_report(earnings_table): # Calculates the total earnings for each week based on the input data.
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table): # Calculates the total earnings for each month based on the input data.
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number): # Returns the name of the month given its number.
    return date(1900, month_number, 1).strftime('%B')

def export_to_excel(df, df_weekly, df_monthly):
    choice = input("Do you want to export the data to an Excel file? (yes/no): ").strip().lower()
    if choice in ('yes', 'y'):
        filename = input("Enter the Excel file name (e.g., 'earnings'): ").strip()
        filename += ".csv"  # Append the .xlsx extension to the filename

        with pd.ExcelWriter(filename) as writer:
            df.to_excel(writer, sheet_name="Daily Earnings", index=False)
            df_weekly.to_excel(writer, sheet_name="Weekly Earnings", index=False)
            df_monthly.to_excel(writer, sheet_name="Monthly Earnings", index=False)

        print(f"Data successfully exported to '{filename}'.")
    else:
        print("Exiting the application.")

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    while True:
        date_input = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")
        else:
            print(f"Using the date: {date_input}")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            # For today or yesterday, ask for daily earnings until today's date
            day_earning = get_day_earnings(date_input)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []
                earnings_table[key].append((day, day_earning))
                date_input += timedelta(days=1)
                if date_input > date.today():
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input)
        else:
            # For past months, ask for day-to-day earnings until the end of the month
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                current_date = date_input
                while current_date < month_end:
                    day_earning = get_day_earnings(current_date)
                    if day_earning is not None:
                        day = current_date.day
                        year, month = current_date.year, current_date.month
                        key = (year, month)
                        if key not in earnings_table:
                            earnings_table[key] = []
                        earnings_table[key].append((day, day_earning))
                    current_date += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    # Convert the dictionary to a pandas DataFrame for better display and sorting
    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    # Sort the DataFrame by the Date column
    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    # Generate weekly report and display it
    weekly_report = generate_weekly_report(earnings_table)
    df_weekly = pd.DataFrame(
        [(f"{year:04d}-Week{week:02d}", earnings) for (year, week), earnings in weekly_report.items()],
        columns=["Week", "Earnings"]
    )

    print("\nWeekly Report:")
    print(df_weekly)

    # Generate monthly report and display it
    monthly_report = generate_monthly_report(earnings_table)
    df_monthly = pd.DataFrame(
        [(f"{year:04d}-{get_month_name(month)}", earnings) for (year, month), earnings in monthly_report.items()],
        columns=["Month", "Earnings"]
    )

    print("\nMonthly Report:")
    print(df_monthly)

    export_to_excel(df, df_weekly, df_monthly)

if __name__ == "__main__":
    main()


Using the date: 2023-07-01

Earnings Table:
          Date  Earnings
0   2023-07-01       0.0
1   2023-07-02       0.0
2   2023-07-03       0.0
3   2023-07-04       0.0
4   2023-07-05       0.0
5   2023-07-06       0.0
6   2023-07-07       0.0
7   2023-07-08       0.0
8   2023-07-09     450.0
9   2023-07-10       0.0
10  2023-07-11      60.0
11  2023-07-12     540.0
12  2023-07-13     360.0
13  2023-07-14     310.0
14  2023-07-15     840.0
15  2023-07-16     500.0
16  2023-07-17       0.0
17  2023-07-18     810.0
18  2023-07-19     630.0
19  2023-07-20     200.0
20  2023-07-21     380.0
21  2023-07-22    1000.0
22  2023-07-23     790.0
23  2023-07-24       0.0
24  2023-07-25      60.0
25  2023-07-26     300.0
26  2023-07-27     300.0
27  2023-07-28     480.0
28  2023-07-29    1140.0
29  2023-07-30     690.0
30  2023-07-31       0.0

Weekly Report:
          Week  Earnings
0  2023-Week26       0.0
1  2023-Week27     450.0
2  2023-Week28    2610.0
3  2023-Week29    3810.0
4  2023-Week30 

In [4]:
import pandas as pd
from datetime import date, timedelta

def get_date_prompt(): #Asks the user for a date input, which can be a specific date, yesterday, today, or 'quit' to exit the application.
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        return input_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")

        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            return yesterday
        elif date_input == "2":
            return today
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date): # Asks the user for earnings for a specific date.
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                return float(day_earning)
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def generate_weekly_report(earnings_table): # Calculates the total earnings for each week based on the input data.
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table): # Calculates the total earnings for each month based on the input data.
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number): # Returns the name of the month given its number.
    return date(1900, month_number, 1).strftime('%B')
def export_to_excel(df, df_weekly, df_monthly,date_input):
    choice = input("Do you want to export the data to an Excel file? (yes/no): ").strip().lower()
    if choice in ('yes', 'y'):
        filename = input("Enter the Excel file name (e.g., 'earnings'): ").strip()
        filename += ".xlsx"  # Append the .xlsx extension to the filename

        try:
            with pd.ExcelFile(filename) as xls:
                existing_df = pd.read_excel(xls, sheet_name="Daily Earnings")

                # Check if the date already exists in the existing DataFrame
                date_column = pd.to_datetime(existing_df["Date"], format="%Y-%m-%d")
                if date_input in date_column.values:
                    overwrite_choice = input("Data for the selected date already exists. Do you want to overwrite it? (yes/no): ").strip().lower()
                    if overwrite_choice in ('yes', 'y'):
                        # Remove the existing entry for the date
                        existing_df = existing_df[~(date_column == date_input)]

                        # Append the new data (yesterday's earnings) to the DataFrame
                        updated_df = pd.concat([existing_df, df], ignore_index=True)
                    else:
                        # User chooses not to overwrite, so return without saving
                        print("Exiting the application without saving changes.")
                        return
                else:
                    # Append the new data (yesterday's earnings) to the DataFrame
                    updated_df = pd.concat([existing_df, df], ignore_index=True)

                # Write the updated DataFrame to the Excel file
                with pd.ExcelWriter(filename) as writer:
                    updated_df.to_excel(writer, sheet_name="Daily Earnings", index=False)
                    df_weekly.to_excel(writer, sheet_name="Weekly Earnings", index=False)
                    df_monthly.to_excel(writer, sheet_name="Monthly Earnings", index=False)

                print(f"Data successfully added to '{filename}'.")
        except FileNotFoundError:
            # If the file doesn't exist, create a new one
            with pd.ExcelWriter(filename) as writer:
                df.to_excel(writer, sheet_name="Daily Earnings", index=False)
                df_weekly.to_excel(writer, sheet_name="Weekly Earnings", index=False)
                df_monthly.to_excel(writer, sheet_name="Monthly Earnings", index=False)

            print(f"Data successfully exported to a new file '{filename}'.")
    else:
        print("Exiting the application.")

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    while True:
        date_input = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")
        else:
            print(f"Using the date: {date_input}")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            # For today or yesterday, ask for daily earnings until today's date
            day_earning = get_day_earnings(date_input)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []
                earnings_table[key].append((day, day_earning))
                date_input += timedelta(days=1)
                if date_input > date.today():
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input)
        else:
            # For past months, ask for day-to-day earnings until the end of the month
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                current_date = date_input
                while current_date < month_end:
                    day_earning = get_day_earnings(current_date)
                    if day_earning is not None:
                        day = current_date.day
                        year, month = current_date.year, current_date.month
                        key = (year, month)
                        if key not in earnings_table:
                            earnings_table[key] = []
                        earnings_table[key].append((day, day_earning))
                    current_date += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    # Convert the dictionary to a pandas DataFrame for better display and sorting
    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    # Sort the DataFrame by the Date column
    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    # Generate weekly report and display it
    weekly_report = generate_weekly_report(earnings_table)
    df_weekly = pd.DataFrame(
        [(f"{year:04d}-Week{week:02d}", earnings) for (year, week), earnings in weekly_report.items()],
        columns=["Week", "Earnings"]
    )

    print("\nWeekly Report:")
    print(df_weekly)

    # Generate monthly report and display it
    monthly_report = generate_monthly_report(earnings_table)
    df_monthly = pd.DataFrame(
        [(f"{year:04d}-{get_month_name(month)}", earnings) for (year, month), earnings in monthly_report.items()],
        columns=["Month", "Earnings"]
    )

    print("\nMonthly Report:")
    print(df_monthly)

    export_to_excel(df, df_weekly, df_monthly, date_input)  # Pass date_input to export_to_excel()

if __name__ == "__main__":
    main()


Using the date: 2023-08-01

Earnings Table:
         Date  Earnings
0  2023-08-01     540.0
1  2023-08-02     630.0

Weekly Report:
          Week  Earnings
0  2023-Week31    1170.0

Monthly Report:
         Month  Earnings
0  2023-August    1170.0
Data successfully exported to a new file '1090.xlsx'.


In [5]:
import pandas as pd
from datetime import date, timedelta

def get_date_prompt(): #Asks the user for a date input, which can be a specific date, yesterday, today, or 'quit' to exit the application.
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        return input_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")

        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            return yesterday
        elif date_input == "2":
            return today
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date): # Asks the user for earnings for a specific date.
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                return float(day_earning)
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def generate_weekly_report(earnings_table): # Calculates the total earnings for each week based on the input data.
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table): # Calculates the total earnings for each month based on the input data.
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number): # Returns the name of the month given its number.
    return date(1900, month_number, 1).strftime('%B')

def export_to_excel(df, df_weekly, df_monthly):
    choice = input("Do you want to export the data to an Excel file? (yes/no): ").strip().lower()
    if choice in ('yes', 'y'):
        filename = input("Enter the Excel file name (e.g., 'earnings'): ").strip()
        filename += ".xlsx"  # Append the .xlsx extension to the filename

        with pd.ExcelWriter(filename) as writer:
            df.to_excel(writer, sheet_name="Daily Earnings", index=False)
            df_weekly.to_excel(writer, sheet_name="Weekly Earnings", index=False)
            df_monthly.to_excel(writer, sheet_name="Monthly Earnings", index=False)

        print(f"Data successfully exported to '{filename}'.")
    else:
        print("Exiting the application.")

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    while True:
        date_input = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")
        else:
            print(f"Using the date: {date_input}")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            # For today or yesterday, ask for daily earnings until today's date
            day_earning = get_day_earnings(date_input)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []
                earnings_table[key].append((day, day_earning))
                date_input += timedelta(days=1)
                if date_input > date.today():
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input)
        else:
            # For past months, ask for day-to-day earnings until the end of the month
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                current_date = date_input
                while current_date < month_end:
                    day_earning = get_day_earnings(current_date)
                    if day_earning is not None:
                        day = current_date.day
                        year, month = current_date.year, current_date.month
                        key = (year, month)
                        if key not in earnings_table:
                            earnings_table[key] = []
                        earnings_table[key].append((day, day_earning))
                    current_date += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    # Convert the dictionary to a pandas DataFrame for better display and sorting
    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    # Sort the DataFrame by the Date column
    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    # Generate weekly report and display it
    weekly_report = generate_weekly_report(earnings_table)
    df_weekly = pd.DataFrame(
        [(f"{year:04d}-Week{week:02d}", earnings) for (year, week), earnings in weekly_report.items()],
        columns=["Week", "Earnings"]
    )

    print("\nWeekly Report:")
    print(df_weekly)

    # Generate monthly report and display it
    monthly_report = generate_monthly_report(earnings_table)
    df_monthly = pd.DataFrame(
        [(f"{year:04d}-{get_month_name(month)}", earnings) for (year, month), earnings in monthly_report.items()],
        columns=["Month", "Earnings"]
    )

    print("\nMonthly Report:")
    print(df_monthly)

    export_to_excel(df, df_weekly, df_monthly)

if __name__ == "__main__":
    main()


KeyboardInterrupt: Interrupted by user

In [2]:
import pandas as pd
import tkinter as tk
from tkinter import messagebox, filedialog
from datetime import date, timedelta

class EarningsTrackerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Earnings Tracker App")

        self.earnings_table = {}
        self.df = None
        self.df_weekly = None
        self.df_monthly = None

        self.create_widgets()

    def create_widgets(self):
        # Create date input frame
        date_frame = tk.Frame(self.root)
        date_frame.pack(padx=10, pady=10)

        self.date_input = tk.IntVar()
        self.date_input.set(2)  # Default value for Today

        self.radio_today = tk.Radiobutton(date_frame, text="Today", variable=self.date_input, value=2)
        self.radio_today.pack(side=tk.LEFT)

        self.radio_yesterday = tk.Radiobutton(date_frame, text="Yesterday", variable=self.date_input, value=1)
        self.radio_yesterday.pack(side=tk.LEFT)

        self.radio_custom = tk.Radiobutton(date_frame, text="Custom Date (YYYY-MM)", variable=self.date_input, value=0)
        self.radio_custom.pack(side=tk.LEFT)

        self.date_entry = tk.Entry(date_frame)
        self.date_entry.pack(side=tk.LEFT)

        self.date_submit_button = tk.Button(date_frame, text="Submit", command=self.get_date_prompt)
        self.date_submit_button.pack(side=tk.LEFT)

        # Create earnings input frame
        earnings_frame = tk.Frame(self.root)
        earnings_frame.pack(padx=10, pady=10)

        self.earnings_prompt_label = tk.Label(earnings_frame, text="Enter earnings:")
        self.earnings_prompt_label.pack(side=tk.LEFT)

        self.earnings_input = tk.StringVar()
        self.earnings_entry = tk.Entry(earnings_frame, textvariable=self.earnings_input)
        self.earnings_entry.pack(side=tk.LEFT)

        self.earnings_submit_button = tk.Button(earnings_frame, text="Submit", command=self.get_day_earnings)
        self.earnings_submit_button.pack(side=tk.LEFT)

        # Create earnings table frame
        table_frame = tk.Frame(self.root)
        table_frame.pack(padx=10, pady=10)

        self.earnings_table_label = tk.Label(table_frame, text="Earnings Table:")
        self.earnings_table_label.pack()

        self.earnings_table_text = tk.Text(table_frame, height=10, width=40)
        self.earnings_table_text.pack()

        # Create export button
        export_button = tk.Button(self.root, text="Export to Excel", command=self.export_to_excel)
        export_button.pack(pady=10)

    def get_date_prompt(self):
        date_input = self.date_input.get()

        if date_input == 0:  # Custom Date option selected
            try:
                year, month = map(int, self.date_entry.get().strip().split("-"))
                input_date = date(year, month, 1)

                # Ensure the date is not in the future
                if input_date > date.today():
                    messagebox.showerror("Invalid input", "Date cannot be in the future.")
                else:
                    self.date_entry.delete(0, tk.END)
                    self.update_earnings_table(input_date)
            except ValueError:
                messagebox.showerror("Invalid input", "Please enter a valid date in the format 'YYYY-MM'.")
        else:  # Today or Yesterday option selected
            self.date_entry.delete(0, tk.END)
            date_selected = date.today() if date_input == 2 else date.today() - timedelta(days=1)
            self.update_earnings_table(date_selected)

    def get_day_earnings(self):
        day_earning = self.earnings_input.get()
        try:
            day_earning = float(day_earning)
            day_date = date.today() if not self.df else self.df.iloc[-1]["Date"]
            day = day_date.day
            year, month = day_date.year, day_date.month
            key = (year, month)
            if key not in self.earnings_table:
                self.earnings_table[key] = []
            self.earnings_table[key].append((day, day_earning))
            self.earnings_input.set("")
            self.update_earnings_table(day_date + timedelta(days=1))
        except ValueError:
            messagebox.showerror("Invalid input", "Please enter a valid number.")

    def update_earnings_table(self, date_input):
        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            while True:
                if date_input > date.today():
                    messagebox.showinfo("Info", "Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = self.get_day_earnings_prompt(date_input)
                if day_earning is not None:
                    day = date_input.day
                    year, month = date_input.year, date_input.month
                    key = (year, month)
                    if key not in self.earnings_table:
                        self.earnings_table[key] = []
                    self.earnings_table[key].append((day, day_earning))
                else:
                    break
                date_input += timedelta(days=1)
        else:
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                current_date = date_input
                while current_date < month_end:
                    day_earning = self.get_day_earnings_prompt(current_date)
                    if day_earning is not None:
                        day = current_date.day
                        year, month = current_date.year, current_date.month
                        key = (year, month)
                        if key not in self.earnings_table:
                            self.earnings_table[key] = []
                        self.earnings_table[key].append((day, day_earning))
                    else:
                        break
                    current_date += timedelta(days=1)
            except ValueError:
                messagebox.showerror("Invalid input", "Please enter a valid number.")

        # Convert the dictionary to a pandas DataFrame for better display and sorting
        self.df = pd.DataFrame(
            [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in self.earnings_table.items() for day, earnings in data],
            columns=["Date", "Earnings"]
        )

        # Sort the DataFrame by the Date column
        self.df.sort_values(by="Date", inplace=True)

        # Display the earnings table in the GUI
        self.earnings_table_text.delete(1.0, tk.END)
        self.earnings_table_text.insert(tk.END, self.df.to_string(index=False))

        # Generate weekly report and display it
        self.generate_weekly_report()
        self.df_weekly = pd.DataFrame(
            [(f"{year:04d}-Week{week:02d}", earnings) for (year, week), earnings in self.weekly_report.items()],
            columns=["Week", "Earnings"]
        )
        self.df_weekly.sort_values(by="Week", inplace=True)

        # Generate monthly report and display it
        self.generate_monthly_report()
        self.df_monthly = pd.DataFrame(
            [(f"{year:04d}-{self.get_month_name(month)}", earnings) for (year, month), earnings in self.monthly_report.items()],
            columns=["Month", "Earnings"]
        )
        self.df_monthly.sort_values(by="Month", inplace=True)

    def get_day_earnings_prompt(self, day_date):
        while True:
            day_earning = self.earnings_input.get()
            if day_earning.lower() == 'done':
                return None
            else:
                try:
                    day_earning = float(day_earning)
                    return day_earning
                except ValueError:
                    messagebox.showerror("Invalid input", "Please enter a valid number or 'done' to finish input.")

    def generate_weekly_report(self):
        # Generate weekly report and display it
        self.weekly_report = {}
        for (year, month), data in self.earnings_table.items():
            for day, earnings in data:
                week_number = date(year, month, day).isocalendar()[1]
                key = (year, week_number)
                if key not in self.weekly_report:
                    self.weekly_report[key] = 0
                self.weekly_report[key] += earnings

    def generate_monthly_report(self):
        # Generate monthly report and display it
        self.monthly_report = {}
        for (year, month), data in self.earnings_table.items():
            key = (year, month)
            if key not in self.monthly_report:
                self.monthly_report[key] = 0
            for day, earnings in data:
                self.monthly_report[key] += earnings

    def get_month_name(self, month_number):
        return date(1900, month_number, 1).strftime('%B')

    def export_to_excel(self):
        if not self.df:
            messagebox.showwarning("No Data", "No data available to export.")
            return

        filename = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])
        if not filename:
            return

        try:
            self.generate_weekly_report()
            self.generate_monthly_report()

            with pd.ExcelWriter(filename) as writer:
                self.df.to_excel(writer, sheet_name="Daily Earnings", index=False)
                self.df_weekly.to_excel(writer, sheet_name="Weekly Earnings", index=False)
                self.df_monthly.to_excel(writer, sheet_name="Monthly Earnings", index=False)

            messagebox.showinfo("Export Success", f"Data successfully exported to '{filename}'.")
        except Exception as e:
            messagebox.showerror("Export Error", f"An error occurred while exporting data: {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = EarningsTrackerApp(root)
    root.mainloop()


In [53]:
pip install os

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
python 3.11 -m pip install --upgrade pip

SyntaxError: invalid syntax (1177698007.py, line 1)

In [3]:
#Doesn't allow appending of data in an existing Excel file with out overwriting it
import pandas as pd
from datetime import date, timedelta
from openpyxl import load_workbook
import os

def get_date_prompt():
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None, None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        print(f"Using the date: {input_date}")
                        update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
                        if update_current_month in ('yes', 'y'):
                            current_date = today
                        else:
                            current_date = None
                        return input_date, current_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")
        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            print(f"Using the date: {yesterday}")
            update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
            if update_current_month in ('yes', 'y'):
                current_date = today
            else:
                current_date = None
            return yesterday, current_date
        elif date_input == "2":
            print(f"Using the date: {today}")
            return today, today  # Allow data entry only up to today's date
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date, current_date):
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                earnings = float(day_earning)
                if current_date and day_date > current_date:
                    print("Cannot input earnings for future dates. Please enter earnings only up to today's date.")
                elif earnings == 0:
                    print("Earnings cannot be zero. Please enter a valid non-zero number or 'done' to finish input.")
                else:
                    return earnings
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    # Prompt for the CSV file name
    csv_file_name = input("Enter the CSV file name (without extension): ").strip()
    csv_file_path = f"{csv_file_name}.csv"

    # Check if the file exists
    if os.path.exists(csv_file_path):
        # If the file exists, read the existing data from the CSV file
        existing_df = pd.read_csv(csv_file_path)
        existing_earnings_table = {}
        for _, row in existing_df.iterrows():
            earnings_date = pd.to_datetime(row["Date"]).date()
            earnings_value = row["Earnings"]
            year, month, day = earnings_date.year, earnings_date.month, earnings_date.day
            key = (year, month)
            if key not in existing_earnings_table:
                existing_earnings_table[key] = []
            existing_earnings_table[key].append((day, earnings_value))
    else:
        # If the file doesn't exist, start with an empty dictionary
        existing_earnings_table = {}

        # Create the CSV file with the required headers
        with open(csv_file_path, "w") as file:
            file.write("Date,Earnings\n")

    while True:
        date_input, current_date = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            day_earning = get_day_earnings(date_input, current_date)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []

                # Check if the date already exists in existing data, if so, skip adding
                if key in existing_earnings_table and any(day == existing_day for existing_day, _ in existing_earnings_table[key]):
                    print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                else:
                    earnings_table[key].append((day, day_earning))

                date_input += timedelta(days=1)
                if current_date and date_input > current_date:
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input, current_date)
        else:
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                while date_input < month_end:
                    if current_date and date_input > current_date:
                        print("Cannot input earnings for future dates. Stopping at today's date.")
                        break

                    if date_input not in earnings_table:
                        day_earning = get_day_earnings(date_input, current_date)
                        if day_earning is not None:
                            day = date_input.day
                            year, month = date_input.year, date_input.month
                            key = (year, month)
                            if key not in earnings_table:
                                earnings_table[key] = []

                            # Check if the date already exists in existing data, if so, skip adding
                            if key in existing_earnings_table and any(day == existing_day for existing_day, _ in existing_earnings_table[key]):
                                print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                            else:
                                earnings_table[key].append((day, day_earning))

                    date_input += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    weekly_report = generate_weekly_report(earnings_table)
    df_weekly = pd.DataFrame(
        [(f"{year:04d}-W{week:02d}", earnings) for (year, week), earnings in weekly_report.items()],
        columns=["Week", "Earnings"]
    )

    print("\nWeekly Report:")
    print(df_weekly)

    monthly_report = generate_monthly_report(earnings_table)
    df_monthly = pd.DataFrame(
        [(f"{year:04d}-{get_month_name(month)}", earnings) for (year, month), earnings in monthly_report.items()],
        columns=["Month", "Earnings"]
    )

    print("\nMonthly Report:")
    print(df_monthly)

    # Export the updated data to the CSV file
    with open(csv_file_path, "a") as file:
        df.to_csv(file, index=False, header=not os.path.exists(csv_file_path))
    with open(csv_file_path, "a") as file:
        df_weekly.to_csv(file, index=False, header=not os.path.exists(csv_file_path))
    with open(csv_file_path, "a") as file:
        df_monthly.to_csv(file, index=False, header=not os.path.exists(csv_file_path))


def generate_weekly_report(earnings_table):
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table):
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number):
    return date(1900, month_number, 1).strftime('%B')

def append_df_to_excel(filename, df, sheet_name, **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename] into [sheet_name]
    """
    try:
        book = load_workbook(filename)
        writer = pd.ExcelWriter(filename, engine='openpyxl')
        writer.book = book
        writer.sheets = {ws.title: ws for ws in book.worksheets}

        if sheet_name in book.sheetnames:
            # Load existing data
            reader = pd.read_excel(filename, sheet_name=sheet_name)
            existing_data = pd.DataFrame(reader)

            if sheet_name == "Daily Earnings":
                # Append new data only if today's data is not already present
                if "Date" in existing_data.columns and date.today().strftime('%Y-%m-%d') not in existing_data["Date"].values:
                    startrow = existing_data.shape[0] + 1
                    df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow, **to_excel_kwargs)
            else:
                # For other sheets, append new data below existing data
                startrow = existing_data.shape[0] + 1
                df.to_excel(writer, sheet_name=sheet_name, index=False, startrow=startrow, **to_excel_kwargs)
        else:
            # If the sheet does not exist, create a new sheet and write the data
            df.to_excel(writer, sheet_name=sheet_name, index=False, **to_excel_kwargs)

        writer.save()
        writer.close()
    except FileNotFoundError:
        # If the file doesn't exist, create a new one
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False, **to_excel_kwargs)

if __name__ == "__main__":
    main()


Using the date: 2023-08-01
Cannot input earnings for future dates. Stopping at today's date.

Earnings Table:
         Date  Earnings
0  2023-08-01     540.0
1  2023-08-02     630.0

Weekly Report:
       Week  Earnings
0  2023-W31    1170.0

Monthly Report:
         Month  Earnings
0  2023-August    1170.0


In [1]:
import pandas as pd
from datetime import date, timedelta
import os
import sqlite3


def get_date_prompt():
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None, None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        print(f"Using the date: {input_date}")
                        update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
                        if update_current_month in ('yes', 'y'):
                            current_date = today
                        else:
                            current_date = None
                        return input_date, current_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")
        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            print(f"Using the date: {yesterday}")
            update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
            if update_current_month in ('yes', 'y'):
                current_date = today
            else:
                current_date = None
            return yesterday, current_date
        elif date_input == "2":
            print(f"Using the date: {today}")
            return today, today  # Allow data entry only up to today's date
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date, current_date):
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                earnings = float(day_earning)
                if current_date and day_date > current_date:
                    print("Cannot input earnings for future dates. Please enter earnings only up to today's date.")
                elif earnings == 0:
                    print("Earnings cannot be zero. Please enter a valid non-zero number or 'done' to finish input.")
                else:
                    return earnings
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def generate_weekly_report(earnings_table):
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table):
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number):
    return date(1900, month_number, 1).strftime('%B')

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    # Prompt for the CSV file name
    csv_file_name = input("Enter the CSV file name (without extension): ").strip()
    csv_file_path = f"{csv_file_name}.csv"

    # Check if the file exists
    if os.path.exists(csv_file_path):
        # If the file exists, read the existing data from the CSV file
        existing_df = pd.read_csv(csv_file_path)
        existing_earnings_table = {}
        for _, row in existing_df.iterrows():
            earnings_date = pd.to_datetime(row["Date"]).date()
            earnings_value = row["Earnings"]
            year, month, day = earnings_date.year, earnings_date.month, earnings_date.day
            key = (year, month)
            if key not in existing_earnings_table:
                existing_earnings_table[key] = []
            existing_earnings_table[key].append((day, earnings_value))
    else:
        # If the file doesn't exist, start with an empty dictionary
        existing_earnings_table = {}

        # Create the CSV file with the required headers
        with open(csv_file_path, "w") as file:
            file.write("Date,Earnings\n")

    while True:
        date_input, current_date = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            day_earning = get_day_earnings(date_input, current_date)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []

                # Check if the date already exists in existing data, if so, skip adding
                if key in existing_earnings_table and any(day == existing_day for existing_day, _ in existing_earnings_table[key]):
                    print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                else:
                    earnings_table[key].append((day, day_earning))

                date_input += timedelta(days=1)
                if current_date and date_input > current_date:
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input, current_date)
        else:
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                while date_input < month_end:
                    if current_date and date_input > current_date:
                        print("Cannot input earnings for future dates. Stopping at today's date.")
                        break

                    if date_input not in earnings_table:
                        day_earning = get_day_earnings(date_input, current_date)
                        if day_earning is not None:
                            day = date_input.day
                            year, month = date_input.year, date_input.month
                            key = (year, month)
                            if key not in earnings_table:
                                earnings_table[key] = []

                            # Check if the date already exists in existing data, if so, skip adding
                            if key in existing_earnings_table and any(day == existing_day for existing_day, _ in existing_earnings_table[key]):
                                print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                            else:
                                earnings_table[key].append((day, day_earning))

                    date_input += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    weekly_report = generate_weekly_report(earnings_table)
    df_weekly = pd.DataFrame(
        [(f"{year:04d}-W{week:02d}", earnings) for (year, week), earnings in weekly_report.items()],
        columns=["Week", "Earnings"]
    )

    print("\nWeekly Report:")
    print(df_weekly)

    monthly_report = generate_monthly_report(earnings_table)
    df_monthly = pd.DataFrame(
        [(f"{year:04d}-{get_month_name(month)}", earnings) for (year, month), earnings in monthly_report.items()],
        columns=["Month", "Earnings"]
    )

    print("\nMonthly Report:")
    print(df_monthly)

    # Export the updated data to the CSV file
    df.to_csv(csv_file_path, index=False, header=not os.path.exists(csv_file_path))
    df_weekly.to_csv(csv_file_path, index=False, header=not os.path.exists(csv_file_path), mode="a")
    df_monthly.to_csv(csv_file_path, index=False, header=not os.path.exists(csv_file_path), mode="a")

if __name__ == "__main__":
    main()


Using the date: 2023-08-01
Cannot input earnings for future dates. Stopping at today's date.

Earnings Table:
         Date  Earnings
0  2023-08-01     540.0
1  2023-08-02     630.0

Weekly Report:
       Week  Earnings
0  2023-W31    1170.0

Monthly Report:
         Month  Earnings
0  2023-August    1170.0


In [62]:
pip install pandas sqlite3

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import date, timedelta
import sqlite3
import os

def get_date_prompt():
    today = date.today()
    while True:
        date_input = input("Enter the date (0 for input, 1 for yesterday, 2 for today, or 'quit' to exit): ").strip().lower()

        if date_input == "quit":
            return None, None
        elif date_input == "0":
            while True:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > today:
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        print(f"Using the date: {input_date}")
                        update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
                        if update_current_month in ('yes', 'y'):
                            current_date = date.today()
                        else:
                            current_date = None
                        return input_date, current_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")
        elif date_input == "1":
            yesterday = today - timedelta(days=1)
            print(f"Using the date: {yesterday}")
            update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
            if update_current_month in ('yes', 'y'):
                current_date = today
            else:
                current_date = None
            return yesterday, current_date
        elif date_input == "2":
            print(f"Using the date: {today}")
            return today, today  # Allow data entry only up to today's date
        else:
            print("Invalid input. Please enter 0, 1, 2, or 'quit'.")

def get_day_earnings(day_date, current_date):
    while True:
        try:
            day_earning = input(f"Enter earnings for {day_date.strftime('%Y-%m-%d')} (or 'done' to finish input): ")
            if day_earning.lower() == 'done':
                return None
            else:
                earnings = float(day_earning)
                if current_date and day_date > current_date:
                    print("Cannot input earnings for future dates. Please enter earnings only up to today's date.")
                elif earnings == 0:
                    print("Earnings cannot be zero. Please enter a valid non-zero number or 'done' to finish input.")
                else:
                    return earnings
        except ValueError:
            print("Invalid input. Please enter a valid number or 'done' to finish input.")

def generate_weekly_report(earnings_table):
    weekly_report = {}

    for (year, month), data in earnings_table.items():
        for day, earnings in data:
            week_number = date(year, month, day).isocalendar()[1]
            key = (year, week_number)
            if key not in weekly_report:
                weekly_report[key] = 0
            weekly_report[key] += earnings

    return weekly_report

def generate_monthly_report(earnings_table):
    monthly_report = {}

    for (year, month), data in earnings_table.items():
        key = (year, month)
        if key not in monthly_report:
            monthly_report[key] = 0
        for day, earnings in data:
            monthly_report[key] += earnings

    return monthly_report

def get_month_name(month_number):
    return date(1900, month_number, 1).strftime('%B')

def main():
    earnings_table = {}  # Initialize the earnings_table dictionary

    # Prompt for the SQLite database file name
    db_file_name = input("Enter the SQLite database file name (without extension): ").strip()
    db_file_path = f"{db_file_name}.db"

    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_file_path)

    # Create the earnings table if it doesn't exist
    create_table_query = """
    CREATE TABLE IF NOT EXISTS earnings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        date DATE NOT NULL,
        earnings FLOAT NOT NULL
    );
    """
    conn.execute(create_table_query)

    # Read existing data from the database and populate the earnings_table dictionary
    query = "SELECT date, earnings FROM earnings;"
    df_from_db = pd.read_sql_query(query, conn)

    # Create the earnings_table dictionary from the fetched data
    for _, row in df_from_db.iterrows():
        earnings_date = pd.to_datetime(row["date"]).date()
        earnings_value = float(row["earnings"])
        year, month, day = earnings_date.year, earnings_date.month, earnings_date.day
        key = (year, month)
        if key not in earnings_table:
            earnings_table[key] = []
        earnings_table[key].append((day, earnings_value))

    while True:
        date_input, current_date = get_date_prompt()

        if date_input is None:
            break

        if date_input == date.today():
            print("Using current date from the system.")
            current_date = date.today()
        elif date_input == date.today() - timedelta(days=1):
            print("Using yesterday's date.")
            current_date = date.today()
        elif date_input == 0:
            # Handle custom date input (0) and reset date_input
            date_input = None
            while date_input is None:
                try:
                    date_str = input("Enter the date (YYYY-MM): ")
                    year, month = map(int, date_str.split("-"))
                    input_date = date(year, month, 1)

                    # Ensure the date is not in the future
                    if input_date > date.today():
                        print("Invalid input. Date cannot be in the future.")
                    else:
                        print(f"Using the date: {input_date}")
                        update_current_month = input("Is this for the current month? (yes/no): ").strip().lower()
                        if update_current_month in ('yes', 'y'):
                            current_date = date.today()
                        else:
                            current_date = None
                        date_input = input_date
                except ValueError:
                    print("Invalid input. Please enter a valid date in the format 'YYYY-MM'.")

        if date_input == date.today() or date_input == date.today() - timedelta(days=1):
            day_earning = get_day_earnings(date_input, current_date)
            while day_earning is not None:
                day = date_input.day
                year, month = date_input.year, date_input.month
                key = (year, month)
                if key not in earnings_table:
                    earnings_table[key] = []

                # Check if the date already exists in existing data, if so, skip adding
                if key in earnings_table and any(day == existing_day for existing_day, _ in earnings_table[key]):
                    print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                else:
                    earnings_table[key].append((day, day_earning))

                date_input += timedelta(days=1)
                if current_date and date_input > current_date:
                    print("Cannot input earnings for future dates. Stopping at today's date.")
                    break
                day_earning = get_day_earnings(date_input, current_date)
        else:
            try:
                month_end = date(date_input.year, date_input.month, 1) + timedelta(days=31)
                while date_input < month_end:
                    if current_date and date_input > current_date:
                        print("Cannot input earnings for future dates. Stopping at today's date.")
                        break

                    if date_input not in earnings_table:
                        day_earning = get_day_earnings(date_input, current_date)
                        if day_earning is not None:
                            day = date_input.day
                            year, month = date_input.year, date_input.month
                            key = (year, month)
                            if key not in earnings_table:
                                earnings_table[key] = []

                            # Check if the date already exists in existing data, if so, skip adding
                            if key in earnings_table and any(day == existing_day for existing_day, _ in earnings_table[key]):
                                print(f"Earnings for {date_input.strftime('%Y-%m-%d')} are already recorded. No need to append.")
                            else:
                                earnings_table[key].append((day, day_earning))

                    date_input += timedelta(days=1)
            except ValueError:
                print("Invalid input. Please enter a valid number.")

    df = pd.DataFrame(
        [(f"{year:04d}-{month:02d}-{day:02d}", earnings) for (year, month), data in earnings_table.items() for day, earnings in data],
        columns=["Date", "Earnings"]
    )

    df.sort_values(by="Date", inplace=True)

    print("\nEarnings Table:")
    print(df)

    # Export the updated data to the database (Append to the existing data)
    df.to_sql("earnings", conn, if_exists="append", index=False)

    # Generate weekly report
    query = """
        SELECT strftime('%Y', date) as year, strftime('%W', date) as week, SUM(earnings) as earnings
        FROM earnings GROUP BY year, week;
    """
    df_weekly_report = pd.read_sql_query(query, conn)

    print("\nWeekly Report:")
    print(df_weekly_report)

    # Generate monthly report
    query = """
        SELECT strftime('%Y', date) as year, strftime('%m', date) as month, SUM(earnings) as earnings
        FROM earnings GROUP BY year, month;
    """
    df_monthly_report = pd.read_sql_query(query, conn)
    conn.close()

    print("\nMonthly Report:")
    print(df_monthly_report)

if __name__ == "__main__":
    main()


Using the date: 2023-08-01
Cannot input earnings for future dates. Stopping at today's date.

Earnings Table:
         Date  Earnings
0  2023-08-01     540.0
1  2023-08-02     630.0
2  2023-08-03     410.0
3  2023-08-04     660.0
4  2023-08-05    1130.0
5  2023-08-06     640.0

Weekly Report:
   year week  earnings
0  2023   31    4010.0

Monthly Report:
   year month  earnings
0  2023    08    4010.0
